In [5]:
import pandas as pd
import numpy as np
import faiss
import torch
from sentence_transformers import SentenceTransformer
from google.colab import drive

In [9]:
df = pd.read_csv('/content/drive/MyDrive/ML recommendation dataset/arxiv_processed.csv')
df.head()

,id,title,abstract,authors,category_code,update_date,clean_title,clean_abstract,category
0,acc-phys/9607001,An Investigation of Stochastic Cooling in the ...,This report provides a description of unbunc...,O. Meincke,acc-phys physics.acc-ph,2008-02-03,an investigation of stochastic cooling in the ...,this report provides a description of unbunche...,"Accelerator Physics, Physics – Accelerator Phy..."
1,acc-phys/9601001,Particle Motion in the Stable Region Near an E...,This paper studies the particle motion when ...,G. Parzen (Brookhaven National Laboratory),acc-phys physics.acc-ph,2008-02-03,particle motion in the stable region near an e...,this paper studies the particle motion when th...,"Accelerator Physics, Physics – Accelerator Phy..."
2,acc-phys/9602001,Muon Colliders,Muon Colliders have unique technical and phy...,"R. B. Palmer(BNL), A. Sessler(LBNL), A. Skrins...",acc-phys physics.acc-ph,2012-08-29,muon colliders,muon colliders have unique technical and physi...,"Accelerator Physics, Physics – Accelerator Phy..."
3,adap-org/9306005,Prediction and Adaptation in an Evolving Chaot...,We describe the results of analytic calculat...,"Alfred H\""ubler and David Pines (Santa Fe Inst...",adap-org chao-dyn nlin.AO nlin.CD,2008-02-03,prediction and adaptation in an evolving chaot...,we describe the results of analytic calculatio...,"Adaptation, Noise, and Self-Organizing Systems..."
4,chao-dyn/9407001,Pattern Dynamics of a Coupled Map Lattice for ...,The pattern dynamics of the one-way coupled ...,Frederick H. Willeboordse (University of Tokyo...,chao-dyn adap-org nlin.AO nlin.CD nlin.PS patt...,2015-06-24,pattern dynamics of a coupled map lattice for ...,the pattern dynamics of the one way coupled lo...,"Adaptation, Noise, and Self-Organizing Systems..."


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194065 entries, 0 to 194064
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   id              194065 non-null  object
 1   title           194065 non-null  object
 2   abstract        194065 non-null  object
 3   authors         194065 non-null  object
 4   category_code   194065 non-null  object
 5   update_date     194065 non-null  object
 6   clean_title     194063 non-null  object
 7   clean_abstract  194065 non-null  object
 8   category        194065 non-null  object
dtypes: object(9)
memory usage: 13.3+ MB


In [11]:
print("FAISS version:", faiss.__version__)

FAISS version: 1.12.0


In [12]:
print("GPU Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))

GPU Available: True
GPU Name: Tesla T4


In [13]:
df["text"] = df["title"].fillna('') + " " + df["abstract"].fillna('')
df = df.dropna(subset=["text"])
df.reset_index(drop=True, inplace=True)
df.head()


,id,title,abstract,authors,category_code,update_date,clean_title,clean_abstract,category,text
0,acc-phys/9607001,An Investigation of Stochastic Cooling in the ...,This report provides a description of unbunc...,O. Meincke,acc-phys physics.acc-ph,2008-02-03,an investigation of stochastic cooling in the ...,this report provides a description of unbunche...,"Accelerator Physics, Physics – Accelerator Phy...",An Investigation of Stochastic Cooling in the ...
1,acc-phys/9601001,Particle Motion in the Stable Region Near an E...,This paper studies the particle motion when ...,G. Parzen (Brookhaven National Laboratory),acc-phys physics.acc-ph,2008-02-03,particle motion in the stable region near an e...,this paper studies the particle motion when th...,"Accelerator Physics, Physics – Accelerator Phy...",Particle Motion in the Stable Region Near an E...
2,acc-phys/9602001,Muon Colliders,Muon Colliders have unique technical and phy...,"R. B. Palmer(BNL), A. Sessler(LBNL), A. Skrins...",acc-phys physics.acc-ph,2012-08-29,muon colliders,muon colliders have unique technical and physi...,"Accelerator Physics, Physics – Accelerator Phy...",Muon Colliders Muon Colliders have unique te...
3,adap-org/9306005,Prediction and Adaptation in an Evolving Chaot...,We describe the results of analytic calculat...,"Alfred H\""ubler and David Pines (Santa Fe Inst...",adap-org chao-dyn nlin.AO nlin.CD,2008-02-03,prediction and adaptation in an evolving chaot...,we describe the results of analytic calculatio...,"Adaptation, Noise, and Self-Organizing Systems...",Prediction and Adaptation in an Evolving Chaot...
4,chao-dyn/9407001,Pattern Dynamics of a Coupled Map Lattice for ...,The pattern dynamics of the one-way coupled ...,Frederick H. Willeboordse (University of Tokyo...,chao-dyn adap-org nlin.AO nlin.CD nlin.PS patt...,2015-06-24,pattern dynamics of a coupled map lattice for ...,the pattern dynamics of the one way coupled lo...,"Adaptation, Noise, and Self-Organizing Systems...",Pattern Dynamics of a Coupled Map Lattice for ...


In [14]:
df.tail()

,id,title,abstract,authors,category_code,update_date,clean_title,clean_abstract,category,text
194060,1506.0457,The Two-envelope Problem: An Informed Choice,The host of a game presents two indistinguis...,Jeffrey Brian Tyler,stat.OT,2015-06-16,the two envelope problem an informed choice,the host of a game presents two indistinguisha...,Statistics – Other Statistics,The Two-envelope Problem: An Informed Choice ...
194061,1904.0433,The Contribution Plot: Decomposition and Graph...,Alzheimer's disease (AD) is a chronic neurod...,"JinCheol Choi, Donghuan Lu, Mirza Faisal Beg, ...",stat.OT,2019-04-10,the contribution plot decomposition and graphi...,alzheimer s disease ad is a chronic neurodegen...,Statistics – Other Statistics,The Contribution Plot: Decomposition and Graph...
194062,2508.09563,Performances and Correlations of Centrality Me...,Numerous centrality measures have been propose...,"Yilin Bi, Xinshan Jiao, Tao Zhou",stat.OT,2025-08-14,performances and correlations of centrality me...,numerous centrality measures have been propose...,Statistics – Other Statistics,Performances and Correlations of Centrality Me...
194063,1410.8868,Precinct Size Matters - The Large Precinct Bia...,Examination of precinct level data in US pre...,Glenn Webb,stat.OT,2014-11-03,precinct size matters the large precinct bias ...,examination of precinct level data in us presi...,Statistics – Other Statistics,Precinct Size Matters - The Large Precinct Bia...
194064,2202.1248,Revisiting the secondary climate attributes fo...,Environmental conditions in various regions ...,"Tao Liao, Paul Kepley, Indraneel Kumar, Samuel...",stat.OT,2022-02-28,revisiting the secondary climate attributes fo...,environmental conditions in various regions ca...,Statistics – Other Statistics,Revisiting the secondary climate attributes fo...


In [15]:
#model = SentenceTransformer('allenai-specter', device='cuda')
model = SentenceTransformer('allenai/specter2_base')

# Encode dataset using the GPU
embeddings = model.encode(
    df["text"].tolist(),
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True,
    device='cuda'
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/754 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Batches:   0%|          | 0/3033 [00:00<?, ?it/s]

In [16]:
embeddings = np.array(embeddings).astype('float32')

# Build index
index = faiss.IndexFlatL2(embeddings.shape[1])  # L2 similarity
index.add(embeddings)

print("✅ FAISS index built with", index.ntotal, "papers.")

✅ FAISS index built with 194065 papers.


In [17]:
def recommend_papers(query, top_k=5):
    # Encode query using same model
    query_emb = model.encode([query], convert_to_numpy=True, device='cuda').astype('float32')

    # Search in FAISS index
    distances, indices = index.search(query_emb, top_k)

    # Show recommended papers
    results = []
    for i, idx in enumerate(indices[0]):
        results.append({
            "rank": i + 1,
            "title": df.iloc[idx]["title"],
            "category": df.iloc[idx]["category"],
            "distance": float(distances[0][i])
        })
    return results


In [21]:
query = "artificial intelligence"
recommendations = recommend_papers(query, top_k=6)

pd.DataFrame(recommendations)

,rank,title,category,distance
0,1,A 2020 taxonomy of algorithms inspired on livi...,"Computer Science – Artificial Intelligence, Co...",103.819397
1,2,Uncertainty Aware AI ML: Why and How,Computer Science – Artificial Intelligence,105.249001
2,3,Modeling of Social Transitions Using Intellige...,Computer Science – Artificial Intelligence,107.740051
3,4,Hybrid Systems Knowledge Representation Using ...,Computer Science – Artificial Intelligence,108.181473
4,5,A Review on the Application of Natural Computi...,"Computer Science – Machine Learning, Computer ...",108.717094
5,6,Comprehensible Artificial Intelligence on Know...,Computer Science – Artificial Intelligence,110.837692
